In [2]:
import datetime
import h5py
import keras
import numpy as np
import os
import pescador
import tensorflow as tf
import time
import sys

sys.path.append("../src")
import localmodule


# Define constants.
dataset_name = localmodule.get_dataset_name()
folds = localmodule.fold_units()
n_input_hops = 104
n_filters = [24, 48, 48]
kernel_size = [5, 5]
pool_size = [2, 4]
n_hidden_units = 64
regularizer = keras.regularizers.l2(0.001)
steps_per_epoch = 1024
epochs = 32
validation_steps = 256
k = 2048
lam = 8.0
batch_size = 32


# Read command-line arguments.
args = ["all", "unit01", "0"]
aug_kind_str = args[0]
unit_str = args[1]
trial_str = args[2]


# Retrieve fold such that unit_str is in the test set.
fold = [f for f in folds if unit_str in f[0]][0]
test_units = fold[0]
training_units = fold[1]
validation_units = fold[2]
    

# Print header.
start_time = int(time.time())
print(str(datetime.datetime.now()) + " Start.")
print("Training Salamon's ICASSP 2017 convnet on " + dataset_name + ". ")
print("Training set: " + ", ".join(training_units) + ".")
print("Validation set: " + ", ".join(validation_units) + ".")
print("Test set: " + ", ".join(test_units) + ".")
print("")
print('h5py version: {:s}'.format(h5py.__version__))
print('keras version: {:s}'.format(keras.__version__))
print('numpy version: {:s}'.format(np.__version__))
print('pescador version: {:s}'.format(pescador.__version__))
print('tensorflow version: {:s}'.format(tf.__version__))
print("")


# Define and compile Keras model.
# NB: the original implementation of Justin Salamon in ICASSP 2017 relies on
# glorot_uniform initialization for all layers, and the optimizer is a
# stochastic gradient descent (SGD) with a fixed learning rate of 0.1.
# Instead, we use a he_uniform initialization for the layers followed
# by rectified linear units (see He ICCV 2015), and replace the SGD by
# the Adam adaptive stochastic optimizer (see Kingma ICLR 2014).
model = keras.models.Sequential()

# Layer 1
bn = keras.layers.normalization.BatchNormalization(
    input_shape=(128, n_input_hops, 1))
model.add(bn)
conv1 = keras.layers.Convolution2D(n_filters[0], kernel_size,
    padding="same", kernel_initializer="he_normal", activation="relu")
model.add(conv1)
pool1 = keras.layers.MaxPooling2D(pool_size=pool_size)
model.add(pool1)

# Layer 2
conv2 = keras.layers.Convolution2D(n_filters[1], kernel_size,
    padding="same", kernel_initializer="he_normal", activation="relu")
model.add(conv2)
pool2 = keras.layers.MaxPooling2D(pool_size=pool_size)
model.add(pool2)

# Layer 3
conv3 = keras.layers.Convolution2D(n_filters[2], kernel_size,
    padding="same", kernel_initializer="he_normal", activation="relu")
model.add(conv3)

# Layer 4
drop1 = keras.layers.Dropout(0.5)
model.add(drop1)
flatten = keras.layers.Flatten()
model.add(flatten)
dense1 = keras.layers.Dense(n_hidden_units,
    kernel_initializer="he_normal", activation="relu",
    kernel_regularizer=regularizer)
model.add(dense1)

# Layer 5
# We put a single output instead of 43 in the original paper, because this
# is binary classification instead of multilabel classification.
drop2 = keras.layers.Dropout(0.5)
model.add(drop2)
dense2 = keras.layers.Dense(1,
    kernel_initializer="glorot_uniform", activation="softmax",
    kernel_regularizer=regularizer)
model.add(dense2)

# Compile model, print model summary.
model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"])
model.summary()

Using TensorFlow backend.


2017-08-10 20:07:01.818740 Start.
Training Salamon's ICASSP 2017 convnet on BirdVox-70k. 
Training set: unit02, unit03, unit05.
Validation set: unit07, unit10.
Test set: unit01.

h5py version: 2.6.0
keras version: 2.0.6
numpy version: 1.13.1
pescador version: 1.0.0
tensorflow version: 1.2.1

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 128, 104, 1)       4         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 104, 24)      624       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 26, 24)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 26, 48)        28848     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 6, 48) 

In [ ]:
import time
# Build Pescador streamers corresponding to log-mel-spectrograms in augmented
# training and validation sets.
training_streamer = localmodule.multiplex_logmelspec(
    aug_kind_str, training_units, n_input_hops, k, lam, batch_size)
validation_streamer = localmodule.multiplex_logmelspec(
    aug_kind_str, validation_units, n_input_hops, k, lam, batch_size)
k = 2048
lam = 8.0
batch_size = 64
print("k = " + str(k))
print("lambda = " + str(lam))
print("batch size = " + str(batch_size))
print("")
for n in range(100):
    start_time = time.time()
    X = next(training_streamer)[0]
    elapsed_time = time.time() - int(start_time)
    elapsed_hours = int(elapsed_time / (60 * 60))
    elapsed_minutes = int((elapsed_time % (60 * 60)) / 60)
    elapsed_seconds = elapsed_time % 60.
    elapsed_str = "{:>02}:{:>02}:{:>05.2f}".format(elapsed_hours,
                                                   elapsed_minutes,
                                                   elapsed_seconds)
    print("Total elapsed time: " + elapsed_str + ".")

2048
8.0
64
Total elapsed time: 00:00:03.44.
Total elapsed time: 00:00:02.64.
Total elapsed time: 00:00:02.40.
Total elapsed time: 00:00:02.18.
Total elapsed time: 00:00:01.75.
Total elapsed time: 00:00:02.00.
Total elapsed time: 00:00:01.30.
Total elapsed time: 00:00:01.60.
Total elapsed time: 00:00:01.86.
Total elapsed time: 00:00:01.94.
Total elapsed time: 00:00:02.13.
Total elapsed time: 00:00:01.25.
Total elapsed time: 00:00:01.43.
Total elapsed time: 00:00:01.62.
Total elapsed time: 00:00:01.59.
Total elapsed time: 00:00:01.74.
Total elapsed time: 00:00:01.68.
Total elapsed time: 00:00:01.94.
Total elapsed time: 00:00:02.04.
Total elapsed time: 00:00:01.29.
Total elapsed time: 00:00:01.27.
Total elapsed time: 00:00:01.16.
Total elapsed time: 00:00:01.28.
Total elapsed time: 00:00:01.22.
Total elapsed time: 00:00:01.33.
Total elapsed time: 00:00:01.43.
Total elapsed time: 00:00:01.38.
Total elapsed time: 00:00:01.44.
Total elapsed time: 00:00:01.63.
Total elapsed time: 00:00:01.40

In [10]:
pescador.__version__

'1.0.0'